<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/NLP2020/02_elementary_Japanese_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 日本語のテキストデータの扱い方

* 日本語は、言語の性質上、文が単語に分たれていない。
* そのため、まず最初に文を単語へ分割する必要がある。
 * 問：分割の単位は、単語である必要があるか?
* この作業を**形態素解析**と言う。
* ここでは、spaCyを介してsudachiというツールで形態素解析をおこなう。
 * 形態素解析システムとしては、他には、MeCabやJUMANが有名。

## 注
* "ContextualVersionConflict ..."というエラーが出たら、上部メニューの「ランタイム」から「ランタイムを再起動」をクリックして、ランタイムを再起動してみてください。

## spaCyの準備

### spaCyを最新版にアップデートする。

In [1]:
! pip install -U spacy

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.3.2)


### バージョンを確認する。

In [2]:
! python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME              MODEL             VERSION                            
package   ja-core-news-sm   ja_core_news_sm   2.3.2   ✔
package   en-core-web-sm    en_core_web_sm    2.3.1   ✔



### 英語の統計モデルもアップデートしておく。
* 今回は英語データは使わないが。

In [3]:
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
! python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME              MODEL             VERSION                            
package   ja-core-news-sm   ja_core_news_sm   2.3.2   ✔
package   en-core-web-sm    en_core_web_sm    2.3.1   ✔



### spaCyの日本語の訓練済み統計モデルをダウンロードする
* 「sm」で終わるものは、最も小さいモデル。
* 「md」で終わるものが、中規模のモデル。
* 「lg」で終わるものが、大規模なモデル。
* 詳細は、以下を参照。
 * https://explosion.ai/blog/spacy-v2-3

In [5]:
! python -m spacy download ja_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('ja_core_news_sm')


### sudachiによる形態素解析を試してみる。
* spaCyの日本語統計モデルをインストールすると、自動的にsudachiがインストールされる。
* そこで、sudachipyコマンドを使って、コマンドラインで形態素解析を試してみる。

In [6]:
! echo "すもももももももものうち" | sudachipy

すもも	名詞,普通名詞,一般,*,*,*	李
も	助詞,係助詞,*,*,*,*	も
もも	名詞,普通名詞,一般,*,*,*	もも
も	助詞,係助詞,*,*,*,*	も
もも	名詞,普通名詞,一般,*,*,*	もも
の	助詞,格助詞,*,*,*,*	の
うち	名詞,普通名詞,副詞可能,*,*,*	内
EOS


## spaCyによる日本語テキスト処理

### 日本語統計モデルをロードする。

In [7]:
from spacy.lang.ja import Japanese

# Load SudachiPy with split mode A (default)
nlp = Japanese()

# Load SudachiPy with split mode B
#cfg = {"split_mode": "B"}
#nlp = Japanese(meta={"tokenizer": {"config": cfg}})

### 文書を作成してみる。

In [8]:
doc = nlp('これは、やっぱり相当冗長な日本語の文じゃないかなと思っていたのですが。')
print(doc.text)

これは、やっぱり相当冗長な日本語の文じゃないかなと思っていたのですが。


### 分かち書きをする。
* 分かち書きされた単語以外の情報も表示させている。
 * tagに「形容動詞」は無く、その語幹が「形状詞」とされている。
 * 「じゃない」の「ない」は、tagでは「形容詞」だが、posでは「AUX」である。

In [9]:
for token in doc:
  print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')

text:これ, pos:PRON, tag:代名詞, lemma:これ
text:は, pos:ADP, tag:助詞-係助詞, lemma:は
text:、, pos:PUNCT, tag:補助記号-読点, lemma:、
text:やっぱり, pos:ADV, tag:副詞, lemma:やっぱり
text:相当, pos:ADV, tag:副詞, lemma:相当
text:冗長, pos:ADJ, tag:形状詞-一般, lemma:冗長
text:な, pos:AUX, tag:助動詞, lemma:だ
text:日本, pos:PROPN, tag:名詞-固有名詞-地名-国, lemma:日本
text:語, pos:NOUN, tag:名詞-普通名詞-一般, lemma:語
text:の, pos:ADP, tag:助詞-格助詞, lemma:の
text:文, pos:NOUN, tag:名詞-普通名詞-一般, lemma:文
text:じゃ, pos:AUX, tag:助動詞, lemma:だ
text:ない, pos:AUX, tag:形容詞-非自立可能, lemma:ない
text:か, pos:PART, tag:助詞-終助詞, lemma:か
text:な, pos:PART, tag:助詞-終助詞, lemma:な
text:と, pos:ADP, tag:助詞-格助詞, lemma:と
text:思っ, pos:VERB, tag:動詞-一般, lemma:思う
text:て, pos:SCONJ, tag:助詞-接続助詞, lemma:て
text:い, pos:AUX, tag:動詞-非自立可能, lemma:いる
text:た, pos:AUX, tag:助動詞, lemma:た
text:の, pos:SCONJ, tag:助詞-準体助詞, lemma:の
text:です, pos:AUX, tag:助動詞, lemma:です
text:が, pos:SCONJ, tag:助詞-接続助詞, lemma:が
text:。, pos:PUNCT, tag:補助記号-句点, lemma:。


# 課題2

* Wikipediaの複数のエントリを、lemmaを半角スペースでつないだ、長い文字列へ変換する。
 * ここでは、コンピュータ科学の様々な分野のエントリを題材として使う。
* scikit-learnのCountVectorizerを使って、各エントリにおける単語の出現頻度からなる特徴ベクトルを得る。
* 特徴ベクトルどうしの類似度を計算し、「人工知能」分野と最も似ている分野がどれかを求める。

## 課題の手順(1)

### エントリをダウンロードして形態素解析を適用する

* 例えば、Wikipediaの「人工知能」エントリをダウンロードする。
 * https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD
* そして、そこに含まれる段落（__`<p>`__タグで囲まれた範囲）を列挙する。
* 各段落のテキストに形態素解析を適用する。
* 形態素解析で得られたlemmaを半角スペースでつないで、エントリ全体をひとつの長い文字列にする。
 * text、つまり、単語に分たれたそのままの文字列を半角スペースでつなぐのではない。
 * lemma（原型に戻したもの）を半角スペースでつなぐこと。
 * posやtagを見て、不要そうな単語を適当に削除してもよい。


In [10]:
# 「人工知能」エントリをダウンロードしてparserのインスタンスを作る。

from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [11]:
# 段落のテキストを取得する。

lines = list()
for para in soup.find_all('p'):
  lines.append(para.text)

In [12]:
# Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

tabu = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
tokens = list()
for line in lines:
  for token in nlp(line):
    pos = token.pos_
    if not pos in tabu:
      print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

text:人工, pos:NOUN, tag:名詞-普通名詞-一般, lemma:人工
text:知能, pos:NOUN, tag:名詞-普通名詞-一般, lemma:知能
text:じん, pos:NOUN, tag:名詞-普通名詞-一般, lemma:じん
text:こうち, pos:NOUN, tag:名詞-普通名詞-形状詞可能, lemma:こうち
text:英, pos:NOUN, tag:名詞-普通名詞-一般, lemma:英
text:artificial intelligence, pos:PROPN, tag:名詞-固有名詞-一般, lemma:artificial intelligence
text:AI, pos:NOUN, tag:名詞-普通名詞-一般, lemma:AI
text:エーアイ, pos:PROPN, tag:名詞-固有名詞-一般, lemma:エーアイ
text:計算, pos:NOUN, tag:名詞-普通名詞-サ変可能, lemma:計算
text:computation, pos:NOUN, tag:名詞-普通名詞-一般, lemma:computation
text:いう, pos:VERB, tag:動詞-一般, lemma:いう
text:概念, pos:NOUN, tag:名詞-普通名詞-一般, lemma:概念
text:コンピュータ, pos:NOUN, tag:名詞-普通名詞-一般, lemma:コンピュータ
text:computer, pos:NOUN, tag:名詞-普通名詞-一般, lemma:Computer
text:いう, pos:VERB, tag:動詞-一般, lemma:いう
text:道具, pos:NOUN, tag:名詞-普通名詞-一般, lemma:道具
text:用い, pos:VERB, tag:動詞-一般, lemma:用いる
text:知能, pos:NOUN, tag:名詞-普通名詞-一般, lemma:知能
text:研究, pos:VERB, tag:名詞-普通名詞-サ変可能, lemma:研究
text:計算, pos:NOUN, tag:名詞-普通名詞-サ変可能, lemma:計算
text:機, pos:NOUN, tag:名詞-普通名詞-助数詞可能, le

In [13]:
# すべてのlemmaを半角スペースでつないで、長い文字列にする。

doc_AI = ' '.join(tokens)
print(doc_AI)

人工 知能 じん こうち 英 artificial intelligence AI エーアイ 計算 computation いう 概念 コンピュータ Computer いう 道具 用いる 知能 研究 計算 機 科学 computer science 一 分野 指す 語 1 言語 理解 推論 問題 解決 知的 行動 人間 代わる コンピューター 行う 技術 2 また 計算 機 コンピュータ よる 知的 情報 処理 システム 設計 実現 関する 研究 分野 する 3 日本 大 百科 全書 ニッポニカ 解説 情報 工学 者 通信 工学 者 佐藤 理史 次 よう 述べる 1 人間 知的 能力 コンピュータ 上 実現 様々 技術 ソフトウェア コンピュータ システム 4 応用 例 自然 言語 処理 機械 翻訳 かな 漢字 変換 構文 解析 等 5 専門 家 推論 判断 模倣 エキスパート システム 画像 データ 解析 特定 パターン 検出 抽出 画像 認識 等 ある 4 1956 年 ダートマス 会議 ジョン マッカーシー よる 命名 現在 記号 処理 用いる 知能 記述 主体 する 情報 処理 研究 アプローチ いう 意味 あい 使う 家庭 用 電気 機械 器具 制御 システム ゲーム ソフト 思考 ルーチン こう 呼ぶ こと ある プログラミング 言語 LISP よる ELIZA いう カウンセラー 模倣 プログラム 人工 無脳 しばしば 引き合い 出す 計算 機 人間 専門 家 役割 する いう エキスパート システム 呼ぶ 研究 情報 処理 システム 実現 人間 暗黙 持つ 常識 記述 問題 なる 実用 利用 困難 視 人工 知能 実現 アプローチ する ファジィ 理論 ニューラル ネットワーク よう アプローチ 知る 従来 人工 知能 gofai good Old fashioned AI 差 記述 記号 明示 性 ある 後 サポート ベクター マシン 注目 集める また 自ら 経験 元 学習 行う 強化 学習 いう 手法 ある 宇宙 おく 知性 最も 強力 形質 レイ カーツ ワイル いう 言葉 通り 知性 機械 表現 実装 いう こと 極めて 重要 作業 2006 年 ディープ ラーニング 深層 学習 登場 2010 年 代 以降 ビッグ データ 登場 よる 一

### 上記の操作をまとめておこなう関数を定義しておく。
* 後で、各エントリについて、この関数を呼び出す。

In [14]:
def morph(soup):

  lines = list()
  for para in soup.find_all('p'):
    lines.append(para.text)

  tabu = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
  tokens = list()
  for line in lines:
    for token in nlp(line):
      pos = token.pos_
      if not pos in tabu:
        tokens.append(token.lemma_)

  return ' '.join(tokens)

## 課題の手順(2)

### 「コンピュータ科学」の様々な分野に関するエントリについて、この作業をしてみる。
*  「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。
* ただし、__`/wiki/`__という文字列で始まっているURLであり、かつ、テンプレートの状態でないものだけを抜き出す。

In [15]:
# 「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。

target_str = '表話編歴コンピュータ科学'
prefix = '/wiki/'

urls = dict()
for table in soup.find_all('table'):
  if table.text[:len(target_str)] != target_str: continue
  for td in table.find_all('td'):
    for a in td.find_all('a'):
      if not a.text: continue
      try:
        if a.text.find('英語版') == -1:
          href = a['href']
          if href[:len(prefix)] == prefix and href.find('/Template:') == -1 and href.find('/Category:') == -1:
            urls[a.text] = 'https://ja.wikipedia.org' + href
      except:
        continue

for k in urls:
  print(k, urls[k])

コンピュータ科学 https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E7%A7%91%E5%AD%A6
ハードウェア https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%BC%E3%83%89%E3%82%A6%E3%82%A7%E3%82%A2
プリント基板 https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%AA%E3%83%B3%E3%83%88%E5%9F%BA%E6%9D%BF
周辺機器 https://ja.wikipedia.org/wiki/%E5%91%A8%E8%BE%BA%E6%A9%9F%E5%99%A8
集積回路 https://ja.wikipedia.org/wiki/%E9%9B%86%E7%A9%8D%E5%9B%9E%E8%B7%AF
Systems on Chip (SoCs) https://ja.wikipedia.org/wiki/System-on-a-chip
エネルギー消費 (グリーン・コンピューティング) https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%AA%E3%83%BC%E3%83%B3IT
EDA https://ja.wikipedia.org/wiki/EDA
ハードウェアアクセラレーション https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%BC%E3%83%89%E3%82%A6%E3%82%A7%E3%82%A2%E3%82%A2%E3%82%AF%E3%82%BB%E3%83%A9%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3
コンピュータ・アーキテクチャ https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E3%83%BB%E3%82%A2%E3%83%BC%E3%82%AD%E3%83%86%E3%82%AF%E3%83%81%E3%83%A3
組み込み

In [16]:
# 各エントリをダウンロードしてparserのインスタンスを作り、辞書として保存。

soups = dict()
for k in urls:
  html = urlopen(urls[k]) 
  soups[k] = BeautifulSoup(html, 'html.parser')

In [17]:
# 先ほど定義した関数morph()を使って各エントリを形態素解析し、lemmaが半角スペースで区切られた文字列へ変換する。

genre = list()
corpus = list()
for k in soups:
  genre.append(k)
  doc = morph(soups[k])
  corpus.append(doc)
  print(k, doc)

コンピュータ科学 計算 機 科学 けい さん かがく 英 computer science コンピュータ 科学 情報 計算 理論 基礎 及び コンピュータ 上 実装 応用 関する 研究 分野 1 2 3 コンピュータ 科学 様々 分野 ある コンピュータ グラフィックス よう 応用 力点 ある 領域 ある 理論 計算 機 科学 呼ぶ 分野 よう 数学 性格 強い 分野 ある 計算 科学 科学 技術 計算 いう 計算 需要 応える ため 分野 それ 実現 手段 研究 高 性能 計算 また 一見 わかる 分類 する 計算 機 工学 ハードウェア プログラミング ソフトウェア いう 分類 ある 再 構成 可能 コンピューティング よう 両方 言える 分野 ある 単純 分類 できる もの ない そろばん アバカス 一種 アナログ 計算 機 言える 機械 いう 計算 手助け もの 古代 存在 計算 機械 今日 言う 機械 する 最初 機械 式 計算 機 ヴィルヘルム シッカート よる 1623 年 作る 4 チャールズ バベッジ ヴィクトリア 朝 時代 プログラム 可能 解析 機関 設計 5 1890 年 ハーマン ホレリス 発明 パンチ カード システム 米国 勢 調査 初めて 使用 6 1920 年 代 以前 Computer いう 言葉 仕事 する 計算 行う 人 計算 手 指す しかし 時代 現代 通じる 計算 理論 計算 模型 考案 クルト ゲーデル アロンゾ チャーチ アラン チューリング 後 計算 機 科学 呼ぶ 分野 先駆 者 計算 可能 性 すなわち 特別 前提 知識 技能 なし 紙 鉛筆 命令 書 もの 計算 興味 抱く 研究 一部 人間 付き物 間違い する こと ない 自動 計算 行う 計算 機械 開発 いう 欲求 基づく もの 重要 洞察 あらゆる 計算 作業 理論 上 全て 実行 可能 汎用 計算 システム 構築 こと 意味 それ 専用 機械 汎用 計算 機 概念 一般 化 汎用 計算 機 いう 概念 創造 現代 計算 機 科学 生む 1940 年 代 入る より 新しい かつ 強力 計算 機 開発 つれる Computer いう 言葉 人間 ない そう いう 機械 指す 言葉 なる 1940 年 代 1950 年 代 かける 次々 電子 式 コン

In [18]:
# 再利用するために、csvファイルとして保存しておく。

import pandas as pd

genre.append('人工知能')
corpus.append(doc_AI)

df = pd.DataFrame(list(zip(genre, corpus)), columns=['genre', 'text'])
print(df.head())
df.to_csv('cs_corpus.csv')

      genre                                               text
0  コンピュータ科学  計算 機 科学 けい さん かがく 英 computer science コンピュータ 科学...
1    ハードウェア  ハードウェア 英語 hardware システム 物理 構成 要素 指す 一般 用語 日本 語...
2    プリント基板  プリント 基板 プリント きばる 短縮 形 pwb PCB 基板 一種 以下 ふた つ まと...
3      周辺機器  周辺 機器 しゅうへん きく また ペリフェラル 英 peripheral コンピュータ ゲ...
4      集積回路  集積 回路 しゅう せき かいろ 英 integrated circuit IC 半 導体 ...


## 課題の手順(3)

### CountVectorizerで単語の出現頻度を要素とするベクトルを作成する。

In [19]:
# 先ほど作成したcsvファイルを読んで、textカラムをCountVectorizerのインスタンスでベクトル化する。
# ベクトルの要素は、各Wikipediaエントリにおける、各単語の出現回数になる。

from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('cs_corpus.csv')

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])

In [20]:
# CountVectorizerで得られるのはsparse matrixである。
print(X)

  (0, 8809)	50
  (0, 8240)	31
  (0, 2551)	1
  (0, 2590)	1
  (0, 2510)	1
  (0, 723)	7
  (0, 2005)	5
  (0, 3532)	18
  (0, 6940)	7
  (0, 8007)	7
  (0, 6352)	1
  (0, 6059)	1
  (0, 6600)	1
  (0, 6903)	4
  (0, 9345)	1
  (0, 8200)	4
  (0, 5832)	12
  (0, 7578)	2
  (0, 2456)	12
  (0, 3433)	2
  (0, 2875)	4
  (0, 5925)	1
  (0, 9485)	5
  (0, 6216)	2
  (0, 7268)	5
  :	:
  (121, 7401)	1
  (121, 9270)	1
  (121, 8855)	1
  (121, 5272)	1
  (121, 5964)	1
  (121, 8228)	1
  (121, 5361)	1
  (121, 77)	1
  (121, 78)	1
  (121, 134)	1
  (121, 79)	1
  (121, 80)	4
  (121, 8299)	1
  (121, 5495)	4
  (121, 6236)	1
  (121, 5408)	1
  (121, 6730)	1
  (121, 83)	1
  (121, 84)	1
  (121, 85)	1
  (121, 6646)	1
  (121, 2556)	1
  (121, 2490)	1
  (121, 7503)	1
  (121, 86)	1


In [21]:
# sparse matrixから、通常のNumPyの配列へ変換する。
X = X.toarray()
print('文書数:{}; 語彙サイズ：{}'.format(*X.shape))

文書数:122; 語彙サイズ：9564


In [22]:
# CountVectorizerによって作られた語彙を取得する。
vocab = vectorizer.get_feature_names()
print(vocab)

['00', '000', '001', '00802', '01', '0160', '0161', '0625', '07', '096', '10', '100', '1000', '1000万', '1000億', '100万', '101', '1010', '1012', '102', '103', '104', '105', '1050億', '106', '107', '108', '1080', '109', '10万', '11', '110', '111', '112', '113', '114', '115', '115億', '116', '117', '118', '119', '12', '120', '1200', '121', '122', '12207', '123', '12345', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '13兆1774億', '14', '140', '1401', '141', '1411', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150億', '151', '152', '153', '154', '15504', '1565927247', '158', '15846', '15億', '16', '16000', '1600億', '1620', '1623', '1642', '1662', '168', '17', '1700', '1715', '1741', '1742', '1748', '1749', '178', '1790', '18', '180', '1830', '1835', '1854', '1860', '1869', '1871', '1874', '1875', '1890', '19', '1901', '1902', '1906', '1908', '1909', '19100', '19123', '1919', '1920', '1922', '1924', '1928', '1

In [23]:
# コーパス全体での出現頻度順で上位20単語を見てみる。

import numpy as np

vocab = np.array(vocab)
print(vocab[np.argsort(- X.sum(axis=0))][:20])

['こと' 'する' 'ある' 'よる' 'いう' 'ソフトウェア' 'システム' 'コンピュータ' 'なる' 'もの' '言語' 'ため'
 '計算' '問題' 'データ' '使う' '情報' '場合' '開発' 'ない']


In [24]:
# 「人工知能」エントリと他のエントリとの距離を求める。

from scipy.spatial import distance

genre = df['genre'].values.tolist()
index_AI = genre.index('人工知能')
print(f'「{genre[index_AI]}」と「{genre[10]}」との間での・・・')
print(f'ユークリッド距離: {np.linalg.norm(X[0] - X[index_AI])}')
print(f'内積: {np.dot(X[0], X[index_AI])}')
print(f'コサイン類似度: {distance.cosine(X[0], X[index_AI])}')

「人工知能」と「組み込みシステム」との間での・・・
ユークリッド距離: 198.4187491140895
内積: 8254
コサイン類似度: 0.5478636045778883
